# MLB Pitchers Multiple Linear Regression - Statsmodels

In [1]:
# import required modules
import pandas as pd
from sklearn import linear_model
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
from sklearn.metrics import classification_report, confusion_matrix
import pandas as pd
from sklearn.metrics import roc_auc_score
from sklearn.metrics import precision_recall_fscore_support
import numpy as np
import statsmodels.api as sm

In [2]:
# load dataset
data = pd.read_csv('../../Capstone/Data/MLB_Ppipeline1.csv', header=0)
data.shape

(31710, 38)

In [4]:
data.columns

Index(['name_common', 'year_ID', 'team_ID', 'lg_ID', 'xRA_sprp_adj',
       'xRA_def_pitcher', 'RS_def_total', 'RpO_replacement', 'teamRpG', 'age',
       'G', 'GS', 'IPouts', 'IPouts_start', 'IPouts_relief', 'RA', 'xRA',
       'PPF', 'PPF_custom', 'xRA_final', 'BIP', 'BIP_perc', 'runs_above_avg',
       'runs_above_avg_adj', 'runs_above_rep', 'GR_leverage_index_avg', 'WAR',
       'oppRpG', 'pyth_exponent', 'waa_win_perc', 'WAA', 'WAA_adj',
       'oppRpG_rep', 'pyth_exponent_rep', 'waa_win_perc_rep', 'WAR_rep',
       'ERA_plus', 'ER_lg'],
      dtype='object')

In [6]:
features=['IPouts', 'xRA', 'xRA_final', 'runs_above_avg', 'runs_above_avg_adj', 'runs_above_rep', 'WAA', 'WAA_adj', 'WAR_rep', 'ER_lg']
X=data[features]
y=data.WAR

In [7]:
# split X and y into training and testing sets
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.25,random_state=2019)

In [8]:
model = sm.OLS(y, X).fit()
predictions = model.predict(X)
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                                 OLS Regression Results                                
=======================================================================================
Dep. Variable:                    WAR   R-squared (uncentered):                   0.988
Model:                            OLS   Adj. R-squared (uncentered):              0.988
Method:                 Least Squares   F-statistic:                          2.659e+05
Date:                Sun, 21 Jun 2020   Prob (F-statistic):                        0.00
Time:                        04:07:08   Log-Likelihood:                          26265.
No. Observations:               31710   AIC:                                 -5.251e+04
Df Residuals:                   31700   BIC:                                 -5.243e+04
Df Model:                          10                                                  
Covariance Type:            nonrobust                                                  
======================================================================================
                         coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------
IPouts                 0.3346      0.007     44.781      0.000       0.320       0.349
xRA                   -0.3818      0.014    -26.360      0.000      -0.410      -0.353
xRA_final              0.0824      0.009      8.861      0.000       0.064       0.101
runs_above_avg        -0.6939      0.010    -70.816      0.000      -0.713      -0.675
runs_above_avg_adj    -0.2309      0.013    -18.421      0.000      -0.255      -0.206
runs_above_rep         0.9756      0.003    318.677      0.000       0.970       0.982
WAA                    0.9553      0.008    115.555      0.000       0.939       0.971
WAA_adj                0.1063      0.001    155.070      0.000       0.105       0.108
WAR_rep                0.0797      0.005     16.805      0.000       0.070       0.089
ER_lg                 -0.0785      0.014     -5.683      0.000      -0.106      -0.051
==============================================================================
Omnibus:                    12837.074   Durbin-Watson:                   1.873
Prob(Omnibus):                  0.000   Jarque-Bera (JB):           198921.740
Skew:                          -1.532   Prob(JB):                         0.00
Kurtosis:                      14.881   Cond. No.                         76.9
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""